# Combined Datasets

In [1]:
# Importing dependencies
import pandas as pd

# Data

Data explanation placeholder

(Talk about original two notebooks?)

In [2]:
# Reading in data
df_movies = pd.read_csv("./Resources/movies_data.csv")
df_economics = pd.read_csv("./Resources/economics_data.csv")

# Combining Data

In [3]:
# Creating a 'Date' for a datetime index
df_movies['Date'] = pd.to_datetime({
    'year': df_movies['released_year'],
    'month': df_movies['released_month'],
    'day': df_movies['released_day']
})

# Setting `Date` as index
df_movies.set_index('Date', inplace=True)

# Ensuring index is sorted with ascending dates
df_movies.sort_index(inplace=True)

In [4]:
# Creating a `Year` and `Month` for concatenation
df_economics['Year'] = df_economics['Date'].str.slice(0,4).astype(int)
df_economics['Month'] = df_economics['Date'].str.slice(5,7).astype(int)

# Renaming to `Year` and `Month` for concatenation
df_movies.rename(columns={
'released_year': 'Year',
'released_month': 'Month'
}, inplace=True)

In [5]:
# Confirming total records before concatenation
print(f'Total ecomonic records: {df_economics.shape[0]}')
print(f'Total movie records: {df_movies.shape[0]}')

Total ecomonic records: 507
Total movie records: 15363


In [6]:
# Combining datasets through concatenation
df_combined = pd.merge(df_economics, df_movies, how='left', on=['Year', 'Month'])

# Confirming total records after concatenation
print(f'Total records: {df_combined.shape[0]}')

Total records: 12188


# EDA

# Train Test Splitting

# Scaling and Econding

# Modeling

Playtime!!

# Eric's Space

# Funda's Space

# Kalvin's Space

# Odele's Space

# Peta's Space

# Vadim's Space

# Findings

# Additional